In [1]:
import pandas as pd
import random
import numpy as np

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
loc = pd.read_csv('loc.csv', sep=";")
prod = pd.read_csv('prod.csv', sep=";")

In [3]:
train = train[train['product_id'].isna() != True]
train = train[train['location_id'].isna() != True]
train = train[train['PRICE_AFTER_DISC'].isna() != True]
train.loc[:, 'location_id'] = train['location_id'].apply(lambda x: int(x))
train.loc[:, 'product_id'] = train['product_id'].apply(lambda x: int(x))
merge_train = train[['demand', 'PROMO1_FLAG',
       'PROMO2_FLAG', 'PRICE_REGULAR', 'PRICE_AFTER_DISC', 'NUM_CONSULTANT',
       'AUTORIZATION_FLAG', 'id']]
train.dropna(inplace=True)

/var/folders/h9/lv_fvvc97912rlw1b4267t8h0000gn/T/ipykernel_11668/4076915570.py:4: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  train.loc[:, 'location_id'] = train['location_id'].apply(lambda x: int(x))
/var/folders/h9/lv_fvvc97912rlw1b4267t8h0000gn/T/ipykernel_11668/4076915570.py:5: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  train.loc[:, 'product_id'] = train['product_id'].apply(lambda x: int(x))


In [4]:
test = test.merge(merge_train, how='left', on=['id'])

In [5]:
#Избавились от колонок с NaN
loc = loc.drop(columns=['STORE_OPEN_DTTM', 'STORE_CLOSURE_DTTM'])
prod = prod.drop(columns=['SALES_INTRODUCTION_DT', 'SALES_DISCONTINUED_DT'])

In [6]:
loc.rename(columns={'STORE_LOCATION_RK':'location_id'}, inplace=True)
prod.rename(columns={'PRODUCT_RK':'product_id'}, inplace=True)

In [7]:
real_train = train.merge(loc, how='left', on='location_id')
real_train = real_train.merge(prod, how='left', on='product_id')

In [8]:
real_test = test.merge(loc, how='left', on='location_id')
real_test = real_test.merge(prod, how='left', on='product_id')

In [9]:
real_test = real_test.drop(columns = ['demand'])

In [10]:
bad_columns_lists = []
for i in real_train.columns:
    if len(real_train[i].unique()) == 1:
        bad_columns_lists.append(i)

real_train.drop(columns=bad_columns_lists, inplace=True)
real_test.drop(columns=bad_columns_lists, inplace=True)

In [11]:
def transform_dataframe(df, date_column):
    transformed_df = df.copy()

    transformed_df[date_column] = pd.to_datetime(transformed_df[date_column])

    transformed_df['day'] = transformed_df[date_column].dt.day
    transformed_df['month'] = transformed_df[date_column].dt.month
    transformed_df['year'] = transformed_df[date_column].dt.year
    transformed_df['day_of_week'] = transformed_df[date_column].dt.weekday
    
    transformed_df.drop(date_column, axis=1, inplace=True)

    return transformed_df

In [12]:
train_data = transform_dataframe(real_train, 'period_dt')
test_data = transform_dataframe(real_test, 'period_dt')

In [13]:
def transform_dt_df(df, column = 'VALID_FROM_DTTM_y'):
    transformed_df = df.copy()

    transformed_df[column] = pd.to_datetime(transformed_df[column], format="%d%b%Y:%H:%M:%S")

    transformed_df[column + '_day'] = transformed_df[column].dt.day
    transformed_df[column + '_month'] = transformed_df[column].dt.month
    transformed_df[column + '_year'] = transformed_df[column].dt.year
    transformed_df[column + '_hour'] = transformed_df[column].dt.hour
    transformed_df[column + '_minute'] = transformed_df[column].dt.minute
    transformed_df[column + '_second'] = transformed_df[column].dt.second
    transformed_df[column + '_weekday'] = transformed_df[column].dt.weekday

    transformed_df.loc[transformed_df[column + '_year'] > 2260, [column + '_day', column + '_month', column + '_year']] = [1, 1, 2260]

    transformed_df.drop(column, axis=1, inplace=True)

    return transformed_df

In [14]:
train_data = transform_dt_df(train_data)
test_data = transform_dt_df(test_data)

In [ ]:
# добавим процент от начальной цены
train_data.loc[:, 'disc_percent'] = train_data['PRICE_AFTER_DISC'] / train_data['PRICE_REGULAR']
test_data.loc[:, 'disc_percent'] = test_data['PRICE_AFTER_DISC'] / test_data['PRICE_REGULAR']

In [ ]:
# Средние значения ответа
group_all = train_data[['month', 'location_id', 'demand']].groupby(by=['month', 'location_id'], as_index=False).mean()
group_date = train_data[['month', 'demand']].groupby(by=['month'], as_index=False).mean()
train_data = train_data.merge(group_all, on=['month', 'location_id'], how='left', suffixes=('', '_all'))
test_data = test_data.merge(group_all, on=['month', 'location_id'], how='left', suffixes=('', '_all'))
train_data = train_data.merge(group_date, on=['month'], how='left', suffixes=('', '_new'))
test_data = test_data.merge(group_date, on=['month'], how='left', suffixes=('', '_new'))

In [ ]:
# добавим номер недели с начала продажи товара
train_data = train_data.sort_values(by=['year', 'month', 'day'])
tmp = train_data.groupby(by=['location_id', 'product_id'], as_index=False).cumcount() + 1
train_data.loc[:, 'number_of_week'] = tmp
test_data = test_data.sort_values(by=['year', 'month', 'day'])
tmp = test_data.groupby(by=['location_id', 'product_id'], as_index=False).cumcount() + 1
test_data.loc[:, 'number_of_week'] = tmp

In [16]:
real_train.to_csv('/Users/ilyas/CS/third_year/ADVB/project/train_add.csv', index=False)
real_test.to_csv('/Users/ilyas/CS/third_year/ADVB/project/test_add.csv', index=False)